In [1]:
import numpy as np
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import express as px
import plotly.graph_objs as go
# ML libraries - idk which one to use yet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_absolute_error, r2_score
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import preprocessing, tree
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import cKDTree


**where i left off (hailey)**
- TA said to use supervised neural network instead of linear regression so I chose MLPRegressor but its lowkey ass (see below)
- I was thinking predicting relative risk at location/hour/weather would be ideal but idk bc the model isn't looking too hot (this would also require requesting hourly nasa data which i didn't fix get_nasa_power_data to change date to hour)

In [2]:
# NASA API
def get_nasa_power_data(lat, lon, start_date, end_date):
    """
    Fetches NASA POWER API data for given latitude, longitude, and time range.

    Args:
    - lat (float): Latitude of the location.
    - lon (float): Longitude of the location.
    - start_date (str): Start date in YYYYMMDD format.
    - end_date (str): End date in YYYYMMDD format.

    Returns:
    - Pandas DataFrame with selected weather parameters.
    """

    # Specify multiple parameters in the API request
    parameters = "PRECSNO,T2MDEW,PRECTOTCORR,T2M,WS2M"

    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "parameters": parameters,
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }

    response = requests.get(url, params=params)

    data = response.json()

    # Convert JSON response to DataFrame and transpose it
    nasa_weather = pd.DataFrame.from_dict(data["properties"]["parameter"], orient="index").T

    # Reset index and rename date column
    nasa_weather.reset_index(inplace=True)
    nasa_weather.rename(columns={"index": "date"}, inplace=True)

    # Convert date column to proper datetime format
    nasa_weather["date"] = pd.to_datetime(nasa_weather["date"], format="%Y%m%d", errors="coerce")
    nasa_weather.dropna(subset=["date"], inplace=True)  # Remove invalid date rows

    nasa_weather.rename(columns={
        "PRECSNO": "Snow_Precipitation",
        "T2MDEW": "Dew_Point_2m",
        "PRECTOTCORR": "Total_Precipitation_mm",
        "T2M": "Temperature_2m_C",
        "WS2M": "Wind_Speed_2m"
    }, inplace=True)
    
    # Add Rounded_Lat and Rounded_Lng for merging
    nasa_weather['Rounded_Lat'] = lat
    nasa_weather['Rounded_Lng'] = lon
    
    # Display DataFrame
    print(f"\n Weather Data for Latitude {lat}, Longitude {lon}\n")
    print(f"\n Weather Data for Latitude {lat}, Longitude {lon}\n")
    nasa_weather['Precipitation(in)'] = nasa_weather['Total_Precipitation_mm'] / 25.4 # mm to in
    nasa_weather['Temperature(F)'] = (nasa_weather['Temperature_2m_C'] * (9./5.)) + 32. # C to F
    nasa_weather['Wind_Speed(mph)'] = nasa_weather['Wind_Speed_2m'] * 2.237 # m/s to mph
    # nasa_weather.dropna()
    display(nasa_weather)  # Works in Jupyter Notebook

    return nasa_weather

In [3]:
# Example: Fetch data for different locations
df_la = get_nasa_power_data(34.05, -118.25, "20240101", "20240107")  # Los Angeles


 Weather Data for Latitude 34.05, Longitude -118.25


 Weather Data for Latitude 34.05, Longitude -118.25



,date,Snow_Precipitation,Dew_Point_2m,Total_Precipitation_mm,Temperature_2m_C,Wind_Speed_2m,Rounded_Lat,Rounded_Lng,Precipitation(in),Temperature(F),Wind_Speed(mph)
0,2024-01-01,0.0,6.12,0.04,12.14,1.64,34.05,-118.25,0.001575,53.852,3.66868
1,2024-01-02,0.0,7.05,0.09,11.55,1.59,34.05,-118.25,0.003543,52.790,3.55683
2,2024-01-03,0.0,6.75,5.87,10.86,3.25,34.05,-118.25,0.231102,51.548,7.27025
3,2024-01-04,0.0,2.14,0.02,9.76,2.89,34.05,-118.25,0.000787,49.568,6.46493
4,2024-01-05,0.0,1.99,0.00,10.80,2.00,34.05,-118.25,0.000000,51.440,4.47400
5,2024-01-06,0.0,1.23,0.12,10.50,2.86,34.05,-118.25,0.004724,50.900,6.39782
6,2024-01-07,0.0,-0.09,0.26,7.77,5.29,34.05,-118.25,0.010236,45.986,11.83373


In [3]:
# inspect us_accident data
us_accidents = pd.read_csv('US_Accidents_March23.csv')
us_accidents.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [5]:
unique_values = us_accidents['Weather_Condition'].unique()

print(unique_values)

['Light Rain' 'Overcast' 'Mostly Cloudy' 'Rain' 'Light Snow' 'Haze'
 'Scattered Clouds' 'Partly Cloudy' 'Clear' 'Snow'
 'Light Freezing Drizzle' 'Light Drizzle' 'Fog' 'Shallow Fog' 'Heavy Rain'
 'Light Freezing Rain' 'Cloudy' 'Drizzle' nan 'Light Rain Showers' 'Mist'
 'Smoke' 'Patches of Fog' 'Light Freezing Fog' 'Light Haze'
 'Light Thunderstorms and Rain' 'Thunderstorms and Rain' 'Fair'
 'Volcanic Ash' 'Blowing Sand' 'Blowing Dust / Windy' 'Widespread Dust'
 'Fair / Windy' 'Rain Showers' 'Mostly Cloudy / Windy'
 'Light Rain / Windy' 'Hail' 'Heavy Drizzle' 'Showers in the Vicinity'
 'Thunderstorm' 'Light Rain Shower' 'Light Rain with Thunder'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'T-Storm'
 'Heavy Thunderstorms and Rain' 'Thunder' 'Heavy T-Storm' 'Funnel Cloud'
 'Heavy T-Storm / Windy' 'Blowing Snow' 'Light Thunderstorms and Snow'
 'Heavy Snow' 'Low Drifting Snow' 'Light Ice Pellets' 'Ice Pellets'
 'Squalls' 'N/A Precipitation' 'Cloudy / Windy' 'Light Fog' 'Sand'
 'Snow G

In [4]:
# list of all columns in US Accident Data
drop_cols = ['ID',
            'Source',
            # 'Severity', # Severity = target column, 1-4, where 1 indicates the least impact on traffic
            'Start_Time',
            'End_Time',
            'Start_Lat',  
            'Start_Lng', 
            'End_Lat',
            'End_Lng',
            'Distance(mi)', # Distance(mi) = target column?, length of road extent affected by accident in miles
            'Description', # Description = human description of accident
            'Street', 
            'City', 
            'County',
            'State',
            'Zipcode',
            'Country',
            'Timezone',
            'Airport_Code',
            'Weather_Timestamp', # Weather_Timestamp = shows time-stamp of weather observation record (in local time)
            # 'Temperature(F)',
            'Wind_Chill(F)',
            'Humidity(%)',
            'Pressure(in)',
            'Visibility(mi)',
            'Wind_Direction',
            # 'Wind_Speed(mph)',
            # 'Precipitation(in)',
            # 'Weather_Condition',
            'Amenity',
            'Bump',
            'Crossing',
            'Give_Way',
            'Junction',
            'No_Exit',
            'Railway',
            'Roundabout',
            'Station',
            'Stop',
            'Traffic_Calming',
            'Traffic_Signal',
            'Turning_Loop',
            'Sunrise_Sunset', # day or night based on sunrise/sunset
            'Civil_Twilight', # day or night based on civil twilight
            'Nautical_Twilight', # day or night based on nautical twilight
            'Astronomical_Twilight'] # day or night based on astronomical twilight

In [5]:
# string feature that if kept will need to be encoded for ML
str_features = 'Weather_Condition'

In [24]:
def prepare_data(df, split, predictors=[], target=[]):
    """
    Prepares the US Accidents DataFrame for merging with NASA weather data, keeping only necessary columns.
    
    Args:
    - df (DataFrame): Raw US Accidents dataset.
    - split (boolean): if true, split df by target and predictor data
    - ml_drop (list): other columns to drop for machine learning model (adjustable if we decide a variable is not good at predicting)
    
    Returns:
    - DataFrame: Processed DataFrame with 'date', 'Rounded_Lat', 'Rounded_Lng', and 'Severity' columns.
    """
    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    # Convert time columns to datetime format
    df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

    # Remove rows with invalid 'Start_Time' values
    df = df[df['Start_Time'].notnull()].copy()

    # Extract 'date' from 'Start_Time' for merging with NASA weather data
    # df['date'] = df['Start_Time'].dt.date

    # Filter for coordinates within LA County
    df = df[(df['Start_Lat'].between(33.7, 34.8)) & (df['Start_Lng'].between(-119.0, -117.6))]

    # Round latitude and longitude to 2 decimal places for approximate matching
    df['Rounded_Lat'] = df['Start_Lat'].round(2)
    df['Rounded_Lng'] = df['Start_Lng'].round(2)
    
    # encode str_features
    le = preprocessing.LabelEncoder()
    df['Weather_Condition'] = le.fit_transform(df['Weather_Condition'])
    
    # add new columns for increased risk with associated conditions and time
    # for now will start by increased risk with associated weather and time
    df['hour'] = df['Start_Time'].astype(str).str.split(' ').str.get(1).str.split(':').str.get(0)
    
    # INCREASED SEVERITY RISK BASED ON LOCATION AND TIME  --------------------------------------------------------------------------
    
    # add col to represent frequency of accidents at this location and time
    df['lt_frequency'] = df.groupby(['Rounded_Lat','Rounded_Lng','hour'])['Severity'].transform('count')

    # add col to represent frequency rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lt_frequency_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','hour'])['lt_frequency'].rank()

    # add col to represent severity rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lt_severity_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','hour'])['Severity'].rank()
    
    # INCREASED SEVERITY RISK BASED ON LOCATION AND WEATHER  --------------------------------------------------------------------------
    
    # add col to represent frequency of accidents at this location and time
    df['lw_frequency'] = df.groupby(['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'])['Severity'].transform('count')

    # add col to represent frequency rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lw_frequency_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'])['lw_frequency'].rank()

    # add col to represent severity rank of accidents at this location and time compared to rest of location and times
    # rank 0 = lowest frequency rank
    df['lw_severity_rank'] = df.groupby(['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'])['Severity'].rank()
    
    # Compute Risk Changes-----------------------------------------------------------------------------------
    
    # Average accident frequency per hour (time baseline)
    avg_hourly_accidents = df.groupby('hour')['lt_frequency'].transform('mean')
    df['time_risk_change'] = (df['lt_frequency'] - avg_hourly_accidents) / avg_hourly_accidents

    # Average accident frequency per weather condition (weather baseline)
    avg_weather_accidents = df.groupby('Weather_Condition')['lt_frequency'].transform('mean')
    df['weather_risk_change'] = (df['lt_frequency'] - avg_weather_accidents) / avg_weather_accidents

    # Combined risk factor (balancing time & weather risks)
    df['combined_risk'] = 0.5 * df['time_risk_change'] + 0.5 * df['weather_risk_change']

    # drop NaN
    df = df.dropna()
    
    # Keep only relevant columns
    df = df.drop(drop_cols, axis=1)
    
    # for machine learning clean and prep
    if split:
        X=df[predictors]
        y=df[target]
        return X,y

    return df

### MLPRegressor

- sklearn documentation https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

Things to consider:

- lowkey don't know good risk metric
- what do we want to predict?
- we could have 2 different models that relative accident frequency based off of location and weather and location and time (hour) --> predicts lw_frequency_rank and lt_frequency_rank (i think predicting relative accident frequency is more intuitive than relative accident severity)
- we could have 2 different models that relative accident severity based off of location and weather and location and time (hour) --> predicts lw_severity_rank and lt_severity_rank
- we could have 4 different models that do both above ^
- we could come up with a new metric that combines all and only have one model (i just don't know best way to represent that)

In [9]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
train, test = train_test_split(us_accidents, test_size=0.3)

In [10]:
# model to predict relative accident frequency based off of location and time
X, y = prepare_data(df=train, split=True, predictors=['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'], target='lw_frequency_rank')

In [11]:
# also takes forever
X_train, X_test, y_train, y_test = train_test_split(X, y)
lt_regr = MLPRegressor(random_state=1, solver='lbfgs',max_iter=3000, tol=0.02, hidden_layer_sizes=200, alpha=0.005, )
lt_regr.fit(X_train, y_train)
lt_regr.predict(X_test)
lt_regr.score(X_test, y_test) # sad scores :(
# increasing max_iter seems to increase the most?

0.24199856265614483

idk... i tried to make it faster by only picking 10 sets of parameters instead of testing everything and not using grid search because that may be making it slower?

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

# Scale features to normilize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define model, settings optimize speed
mlp = MLPRegressor(solver='adam', 
                   activation='relu', 
                   batch_size=128, 
                   early_stopping=True, # prevents unnecessary training
                   random_state=1)

# Define parameter space
param_dist = {
    "max_iter": [1000, 2000, 3000],
    "hidden_layer_sizes": [(100,), (200,), (300,)],
    "alpha": [0.0001, 0.001, 0.005],
    "tol": [0.1, 0.2, 0.3]
}

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(mlp, param_distributions=param_dist, n_iter=10, cv=3, verbose=2, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# Best model
best_params = random_search.best_params_
print("Best parameters:", best_params)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'tol': 0.2, 'max_iter': 1000, 'hidden_layer_sizes': (300,), 'alpha': 0.001}


In [13]:
# ideally once best parameters are found and target prediction is confirmed we just predict whatever using nasa power data

Accident frequency depends more on time and location (e.g., rush hour, high-traffic areas), while severity is influenced by weather conditions and road characteristics (e.g., rain, fog, wind speed). By using two separate models, one for frequency and one for severity, we can train each model with the most relevant features, leading to more accurate predictions.

But this also takes forever to run so im not sure...

In [14]:
# MODEL 1: Predicting Accident Frequency
# learns from past accident data to preduct how frequently accidents occur at a given
# time and location
# select input features and target
predictors_freq = ['Rounded_Lat', 'Rounded_Lng', 'hour']
target_freq = 'lt_frequency'

# Prepare the data
X_freq, y_freq = prepare_data(train, split=True, predictors=predictors_freq, target=target_freq)

#Split into training and test sets
X_train_freq, X_test_freq, y_train_freq, y_test_freq = train_test_split(X_freq, y_freq, test_size=0.2, random_state=42)

# Scale features
scaler_freq = MinMaxScaler()
X_train_freq_scaled = scaler_freq.fit_transform(X_train_freq)
X_test_freq_scaled = scaler_freq.transform(X_test_freq)

# Train Neural Network to predict accident frequency
freq_model = MLPRegressor(hidden_layer_sizes=(64, 32), 
                          max_iter=2000, solver='adam', 
                          activation='relu', 
                          batch_size=256, 
                          early_stopping=True, 
                          random_state=1)

# Train the model
freq_model.fit(X_train_freq_scaled, y_train_freq)

# Test on unseen Data
y_pred_freq = freq_model.predict(X_test_freq_scaled)

# Print Accuracy Results
print("Frequency Model MAE:", mean_absolute_error(y_test_freq, y_pred_freq))
print("Frequency Model score:", r2_score(y_test_freq, y_pred_freq))

# MODEL 2: Predicting Accident Severity
# select features and target for severity model
predictors_sev = ['Rounded_Lat', 'Rounded_Lng', 'hour']
target_sev = 'lt_severity_rank'

# Prepare data
X_sev, y_sev = prepare_data(train, split=True, predictors=predictors_sev, target=target_sev)

# Split into training and testing set
X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(X_sev, y_sev, test_size=0.2, random_state=42)

# Scale features
scaler_sev = MinMaxScaler()
X_train_sev_scaled = scaler_sev.fit_transform(X_train_sev)
X_test_sev_scaled = scaler_sev.transform(X_test_sev)

# Train Neural Network for Severity Prediction
sev_model = MLPRegressor(hidden_layer_sizes=(64, 32), 
                         max_iter=2000, 
                         solver='adam', 
                         activation='relu', 
                         batch_size=256, 
                         early_stopping=True, 
                         random_state=1)

# Train model
sev_model.fit(X_train_sev_scaled, y_train_sev)

# Test model
y_pred_sev = sev_model.predict(X_test_sev_scaled)

# Print Accuracy
print("Severity Model - MAE:", mean_absolute_error(y_test_sev, y_pred_sev))
print("Severity Model - R²:", r2_score(y_test_sev, y_pred_sev))


Frequency Model MAE: 13.901943663841028
Frequency Model score: 0.3365396492233501
Severity Model - MAE: 5.283131020659158
Severity Model - R²: 0.3410470146070598


I got your model to run too...finally haha

In [ ]:
# this takes forever maybe don't run it
regr = MLPRegressor(random_state=1, solver='lbfgs')
parameters = {"max_iter": list(range(2000,5000,1000)), 
              "hidden_layer_sizes": list(range(100, 500, 100)),
             "tol": [0.1,0.2,0.3,0.4,0.5],
             "alpha": [0.0001, 0.0005, 0.001, 0.005, 0.01],}
cv_scores = [] #holds the cross validation scores
#try every combination of estimators and max depth, recording them and their scores
for mi in parameters['max_iter']:
    for hls in parameters['hidden_layer_sizes']:
        for t in parameters['tol']:
            for a in parameters['alpha']:
                regr.set_params(solver='lbfgs', max_iter=mi, hidden_layer_sizes=hls, tol=t, alpha=a)
                scores = cross_val_score(regr, X_train, y_train, cv=5) #gets cross validation score
                
                cv_scores.append({"max_iter":mi,
                                 "hidden_layer_sizes":hls,
                                 "tol":t,
                                 "alpha":a})

best_params = max(cv_scores, key=lambda x: x["mean_accuracy"]) #find the parameters with the highest (best) cv score
best_params

In [ ]:
print(best_params)

In [ ]:
# Extract best parameters
best_max_iter = best_params["max_iter"]
best_hidden_layers = best_params["hidden_layer_sizes"]
best_tol = best_params["tol"]
best_alpha = best_params["alpha"]

# Train final model using best parameters
best_model = MLPRegressor(solver='lbfgs', 
                          max_iter=best_max_iter, 
                          hidden_layer_sizes=best_hidden_layers, 
                          tol=best_tol, 
                          alpha=best_alpha,
                          random_state=1)

best_model.fit(X_train, y_train)

# Evaluate final model
y_pred = best_model.predict(X_test)

print("Final Model - MAE:", mean_absolute_error(y_test, y_pred))
print("Final Model - R²:", r2_score(y_test, y_pred))


I think we should ask how to make this faster because...it took me like 30 minutes to run all of this or more

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn import preprocessing, svm 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) 
linRG = LinearRegression()
linRG.fit(X_train, y_train)
linRG.predict(X_test)
linRG.score(X_test, y_test) 

Trying new neural network based off of https://www.kaggle.com/code/kelixirr/us-accidents-severity-prediction-end-to-end#Preparing-Our-Data-For-The-Model

(i had to download tensorflow)

In [52]:
from __future__ import absolute_import, division, print_function
import os
import sys
# print("Python version:", sys.version)
import tensorflow as tf
# print("TensorFlow version:", tf.__version__)
# keras = tf.keras

In [130]:
# usually only outputs error if y_test has wrong values
from sklearn.model_selection import train_test_split

train, test = train_test_split(us_accidents, test_size=0.3)
# model to predict relative accident frequency based off of location and time
# X, y = prepare_data(df=train, split=True, predictors=['Rounded_Lat','Rounded_Lng','Temperature(F)','Wind_Speed(mph)','Precipitation(in)'], target='Severity')
df_final = prepare_data(train, split=False)
X = df_final.drop(columns=['Severity'])
y = df_final['Severity']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42, stratify=y_temp)  # 0.4 * 25% = 10%

print(f"Training Set Size: {len(X_train)}")
print(f"Validation Set Size: {len(X_val)}")
print(f"Test Set Size: {len(X_test)}") 

Training Set Size: 26161
Validation Set Size: 5232
Test Set Size: 3489


In [131]:
unique_values = y_test.unique()

print(unique_values)
# should be between [0,4]

[2 4]


In [104]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

y_train = y_train - 1
y_val = y_val - 1  

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_scaled, y_val))

train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)
val_dataset = val_dataset.batch(64)

In [105]:
unique_values = train['Severity'].unique()

print(unique_values) # 4 unique class for accident severity

[2 3 4 1]


In [115]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler


model = Sequential()
model.add(Input(shape = (X_train_scaled.shape[1],)))
model.add(Dense(256, activation='relu')) 
model.add(BatchNormalization()) 
model.add(Dropout(0.3)) 
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(4, activation='softmax'))  # 4 classes for accident severity

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 256)            │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,636 (193.89 KB)

 Trainable params: 48,676 (190.14 KB)

 Non-trainable params: 960 (3.75 KB)

In [116]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

def lr_schedule(epoch):
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 10
    lr = initial_lr * (drop ** np.floor((1+epoch)/epochs_drop))
    return lr

checkpoint_path = 'best_model.keras'
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
lr_scheduler = LearningRateScheduler(lr_schedule)


history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=val_dataset,
                    callbacks=[checkpoint, early_stopping, lr_scheduler])

Epoch 1/50
1946/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8349 - loss: 0.5233
Epoch 1: val_loss improved from inf to 0.02986, saving model to best_model.keras
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8366 - loss: 0.5184 - val_accuracy: 0.9913 - val_loss: 0.0299 - learning_rate: 0.0010
Epoch 2/50
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9895 - loss: 0.0387
Epoch 2: val_loss improved from 0.02986 to 0.02591, saving model to best_model.keras
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9895 - loss: 0.0387 - val_accuracy: 0.9914 - val_loss: 0.0259 - learning_rate: 0.0010
Epoch 3/50
1954/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9905 - loss: 0.0324
Epoch 3: val_loss improved from 0.02591 to 0.02400, saving model to best_model.keras
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9906 - loss: 0.0324 - val_accuracy: 0.9915 - val_loss: 0.0240 - learning_rate: 0.0010
Epoch 4/50
1952/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 

In [117]:
os.path.isfile('best_model.keras') # make sure true before loading saved models in later cells

True

In [118]:
# model = tf.keras.models.load_model('/kaggle/working/best_model.keras')
model = tf.keras.models.load_model('best_model.keras')

history = model.fit(train_dataset,
    validation_data=val_dataset,
    initial_epoch=50, 
    epochs=150,
    callbacks=[checkpoint, early_stopping, lr_scheduler],
    verbose=1
)

Epoch 51/150
1974/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9942 - loss: 0.0173
Epoch 51: val_loss did not improve from 0.01593
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9942 - loss: 0.0173 - val_accuracy: 0.9947 - val_loss: 0.0167 - learning_rate: 3.1250e-05
Epoch 52/150
1959/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9944 - loss: 0.0168
Epoch 52: val_loss did not improve from 0.01593
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9944 - loss: 0.0168 - val_accuracy: 0.9948 - val_loss: 0.0166 - learning_rate: 3.1250e-05
Epoch 53/150
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9946 - loss: 0.0169
Epoch 53: val_loss did not improve from 0.01593
1975/1975 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9946 - loss: 0.0169 - val_accuracy: 0.9948 - val_loss: 0.0169 - learning_rate: 3.1250e-05
Epoch 54/150
1954/1975 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9944 - loss: 0.0169
Epoch 54: val_loss did not improve from 0.01593
1975/19

In [119]:
nn_model = tf.keras.models.load_model("best_model.keras")
val_loss, val_accuracy = nn_model.evaluate(val_dataset) 
print(f"Validation Accuracy: {val_accuracy}")

395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9943 - loss: 0.0168
Validation Accuracy: 0.9946177005767822


In [132]:
y_test = y_test - 1
print(y_test) # why are y_test values < 0
X_test_scaled = scaler.transform(X_test)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test)) 
test_dataset = test_dataset.batch(64)

4811478    1
6254359    1
4043068    1
4873285    1
5232055    1
          ..
4299844    1
4780388    1
4406789    1
5612832    1
5612758    1
Name: Severity, Length: 3489, dtype: int64


In [133]:
test_loss, test_accuracy = nn_model.evaluate(test_dataset) 


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9965 - loss: 0.0301  


In [134]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = np.argmax(nn_model.predict(X_test_scaled), axis=1) 
# y_pred = np.argmax(model.predict(X_test_scaled), axis=1) 
print(classification_report(y_test, y_pred))

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3475
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00        14

    accuracy                           1.00      3489
   macro avg       0.33      0.33      0.33      3489
weighted avg       0.99      1.00      0.99      3489



C:\Users\littl\anaconda3\envs\PIC16B-25W\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\littl\anaconda3\envs\PIC16B-25W\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\littl\anaconda3\envs\PIC16B-25W\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\littl\